In [2]:
#import packages 

from glob import glob
import pandas as pd
import os
import shutil
import math

folder_path="./dataset/subset/"
folders = glob(folder_path+'complete/*')

train_split=0.7
no_of_images=0

In [14]:
#split data to training and testing

def createFolder(path):
	if os.path.exists(path):
		shutil.rmtree(path)
	os.makedirs(path)

createFolder(folder_path+"train")
createFolder(folder_path+"test")

for folder in folders:
    images = glob(folder+'/*.jpg')
    name = folder.split('/')[-1]
    createFolder(folder_path+"train/"+name)
    createFolder(folder_path+"test/"+name)
    train_images=math.floor(train_split*len(images))
    for i,image in enumerate(images):
        if(i<=train_images-1):
            shutil.copy(image,folder_path+"train/"+name)
        else:
            shutil.copy(image,folder_path+"test/"+name)
        no_of_images+=1
    print(name)


complete\BabyHat
complete\BabyPants
complete\BabyShirt
complete\womencasualshoes
complete\womenlongsleevetop


In [4]:
# Importing the Keras libraries and packages
from keras.models import Sequential,save_model,load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers.core import Dense, Dropout
from keras.applications import VGG16,Xception,ResNet50,nasnet
from keras.callbacks import ModelCheckpoint,EarlyStopping, TensorBoard

# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.2))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.2))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 5, activation = 'softmax'))
# Compiling the CNN
classifier.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])
classifier.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
__________

In [ ]:

# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255 )

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(folder_path+"train/",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(folder_path+"test/",
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

tensorboard=[TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32, write_graph=True, 
                      write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, 
                      embeddings_metadata=None)]

checkpointer = ModelCheckpoint(filepath='output_models', verbose=1, save_best_only=True)

classifier.fit_generator(training_set,steps_per_epoch = 12500/32,
                         epochs = 15,
                         validation_data = test_set,
                         validation_steps = 30,
                         callbacks =tensorboard)

Found 8751 images belonging to 5 classes.
Found 3753 images belonging to 5 classes.
Epoch 1/15
